# BPE

---

## 一、OOV 问题的核心挑战

**OOV（Out-of-Vocabulary，词汇库外）问题**是自然语言处理（NLP）的核心痛点，指模型在推理或生成过程中遇到 **训练数据中未出现的新词或生僻词**，导致无法正确识别或处理这些词汇的问题。

### **典型场景**
- **新词涌现**：网络热词（如“内卷”“躺平”）、专业术语（如“元宇宙”）等不断产生；
- **低频词问题**：训练语料中低频词（如“unhappiness”）未被纳入词表；
- **形态多样性**：词形变化（如“run”“running”“runner”）增加词表复杂度；
- **多语言差异**：通用词表难以覆盖不同语言的构词规则。

---

## 二、传统分词方案的局限性

| **分词方案**         | **优点**                                                                 | **缺点**                                                                 | **适用场景**                     |
|----------------------|--------------------------------------------------------------------------|--------------------------------------------------------------------------|----------------------------------|
| **字符级**           | 词汇表极小（如英文 26 字符 + 符号），无未登录词（OOV）                   | 序列过长（语义单位破碎），模型学习效率低                                 | 小语种、低资源语言               |
| **词级**             | 语义单位完整，人类可解释性强                                             | 词汇表极大（如英文超百万），OOV 严重（生僻词、缩写、新词）               | 简单任务（如情感分析）、高资源语言 |
| **子词级（如 BPE）** | 词汇表大小适中，OOV 少，语义单位更合理（如 “unhappiness” 拆为 “un-happi-ness”） | 需预先训练分词器，对低频子词仍可能处理不足                               | 预训练模型（BERT/GPT）、多语言任务 |

---

## 三、BPE 的核心原理

BPE（Byte Pair Encoding）通过 **迭代合并高频字符对**，逐步生成子词单元，平衡词汇表大小与语义表达精度：

1. **初始单位**：以字符为最小单位（如英文初始词汇表为所有字母、数字、标点符号）；
2. **迭代合并**：扫描文本，统计相邻字符对的频率，合并最高频的对；
3. **终止条件**：达到预设的词汇表大小或合并次数阈值。

**效果**：
- **高频词**直接合并为完整词（如“the”“and”）；
- **中频词**拆为少量子词（如“apple”→“app”+“le”）；
- **低频词**拆为更多子词（如“unhappiness”→“un”+“happi”+“ness”），显著减少 OOV。

---

## 四、BPE 与 Zipf 定律的关系

### **Zipf 定律简介**
- **定义**：自然语言中，词频与词的排序满足幂律分布：
  $$
  \text{频率} \propto \frac{1}{\text{词序}}
  $$
- **现象**：高频词数量少（如“the”“be”），低频词数量多（如“unhappiness”）。

![Zipf Law示意图](img/4_1_zipfs-law.png)

### **BPE 如何利用 Zipf 定律**
- **高频词优先合并**：通过合并高频字符对（如“the”“and”），快速构建常用词；
- **低频词拆分处理**：低频词因 Zipf 定律天然稀疏，BPE 通过子词单元（如“un-happi-ness”）减少 OOV；
- **词汇表压缩**：Zipf 定律表明高频词贡献主要信息量，BPE 通过控制词汇表大小（如 30k~50k）保留高频词，舍弃低频词，实现高效建模。

---

## 五、BPE 的执行流程（简要）

### **步骤 1：准备训练数据**
训练文本：
```
"low low low lowly lower newer newer"
```
预处理后：
- `low</w>`: 3 次
- `lowly</w>`: 1 次
- `lower</w>`: 1 次
- `newer</w>`: 2 次

---

### **步骤 2：统计子词对频率**
| **子词对**       | **总频率** |
|------------------|------------|
| `(l,o)`          | 5          |
| `(o,w)`          | 5          |
| `(w,e)`          | 3          |
| `(e,r)`          | 3          |
| `(r,</w>)`       | 3          |

---

### **步骤 3：迭代合并子词对**
1. **第一次合并**：`(l,o)` → `lo`
   - `low</w>` → `lo + w + </w>`
2. **第二次合并**：`(lo,w)` → `low`
   - `low</w>` → `low + </w>`
3. **第三次合并**：`(low,</w>)` → `low</w>`
   - `low</w>` → `low</w>`
4. **第四次合并**：`(e,r)` → `er`
   - `lower</w>` → `low + e + er + </w>`

---

## 六、BPE 的解码过程

BPE 解码是 **分词的逆过程**，规则如下：
1. **直接拼接**所有子词；
2. **遇到终止符 `</w>`** 时，替换为空格；
3. **去除多余空格**，得到原始句子。

**示例**：
- 子词序列 `[low</w>, low, l, y</w>]`
  解码后：`low lowly`

---

## 七、BPE 的优缺点总结

### **优点**
- **控制词汇表大小**：避免词级分词的 “百万级词汇表”，降低模型参数量和训练成本；
- **减少 OOV 问题**：低频词、新词可拆分为已有子词，几乎无未登录词；
- **语义连贯性**：合并的子词通常具有语义关联（如 “un-”“-ness”），帮助模型学习词法规律；
- **多语言适配**：无需针对不同语言设计特殊分词规则（如中文可直接以字符为初始单位）。

### **缺点**
- **依赖训练数据**：若训练数据覆盖不足，可能生成不合理的子词（如低频词拆分为过细的字符）；
- **固定合并规则**：一旦训练完成，分词规则固定，无法动态适应新领域的词汇（需重新训练分词器）；
- **处理效率**：长文本分词时，需遍历子词对统计频率，效率低于词级分词。

## 九、BPE代码实现
### 9.1  Learn a variable-length encoding of the vocabulary in a text

In [21]:
from __future__ import unicode_literals

import os
import sys
import inspect
import codecs
import re
import copy
import warnings
from collections import defaultdict, Counter

In [22]:
def update_vocabulary(vocab, file_name, is_dict=False):
    """
    统计文本文件中的词汇频率，更新词汇表字典。

    参数:
        vocab (dict 或 defaultdict(int)): 待更新的词汇表。若为普通字典，新单词默认计数为1。
        file_name (str): 输入文件路径。
        is_dict (bool): 若为 True，每行格式为 "word count"；若为 False，每行按空白分割单词。

    返回:
        defaultdict(int): 更新后的词汇表（键：单词，值：频率计数）。
    """
    # 转换为 defaultdict(int) 以兼容普通字典输入
    if not isinstance(vocab, defaultdict):
        vocab = defaultdict(int, vocab)

    # 使用 utf-8-sig 自动处理 BOM
    with open(file_name, 'r', encoding='utf-8-sig') as fobj:
        for line_num, line in enumerate(fobj, 1):  # 行号从1开始
            line = line.strip('\r\n')  # 去除首尾换行符
            if not line:
                continue  # 跳过空行

            if is_dict:
                parts = line.split()
                if len(parts) != 2:
                    print(f"警告：第 {line_num} 行格式错误（预期 'word count'）: {line}")
                    continue
                word, count_str = parts
                try:
                    count = int(count_str)
                except ValueError as e:
                    print(f"警告：第 {line_num} 行计数转换失败: {count_str}。错误: {e}")
                    continue
                vocab[word] += count
            else:
                # 按任意空白分割单词（处理多空格、制表符等）
                words = line.split()
                for word in words:
                    vocab[word] += 1

    return vocab


def update_pair_statistics(pair, changed, stats, indices):
    """
    更新符号对的索引和频率
    :param pair: (str,str) 当前要合并的符号对（‘a’,b）
    :param changed: List[(j,word,old_word,freq)]
    :param stats: defaultdict(int) (key:符号对，val：频率)
    :param indices: 符号对索引字典:记录该符号对在哪些单词中出现及其次数）。
    :return:
    """
    # step 1: 初始化当前符号对的统计信息
    stats[pair] = 0
    indices[pair] = defaultdict(int)

    # step 2: 分解当前符号对
    first, second = pair
    new_pair = first + second

    # step 3 :遍历所有受影响的单词
    for j, word, old_word, freq in changed:
        # step 3.1: 处理旧单词
        i = 0
        try:
            i = old_word.index(first, i)
        except ValueError as e:
            break
        if i < len(old_word) - 1 and old_word[i + 1] == second:
            # 处理前一个符号对（如 A B → 合并 B C 后，A B 的频率减少）
            if i:
                prev = old_word[i - 1:i + 1]
                stats[prev] -= freq
                indices[prev][j] -= 1
            # 处理后一个符号对（如 B C B → 合并 B C 后，C B 的频率减少）
            if i < len(old_word) - 2:
                if old_word[i + 2] != first or i >= len(old_word) - 3 or old_word[i + 3] != second:
                    nex = old_word[i + 1:i + 3]
                    stats[nex] -= freq
                    indices[nex][j] -= 1

            i += 2
        else:
            i += 1

        i = 0
        # step 4 :处理新词
        while True:
            try:
                i = word.index(new_pair, i)
            except ValueError as e:
                break
            # 处理前一个符号对（如 A BC → 合并 B C 后，A BC 的频率增加）
            if i:
                prev = word[i - 1, i + 1]
                stats[prev] += freq
                indices[prev][j] += 1
            # 处理后一个符号对（如 BC B → 合并 B C 后，BC B 的频率增加）
            if i < len(word) - 1 and word[i + 1] != new_pair:
                nex = word[i:i + 2]
                stats[nex] += freq
                indices[nex][j] += 1
            i += 1


def get_pair_statistic(vocab):
    """
    统计符号对频率和位置
    :param vocab:
    :return:
    """
    # 符号对频率统计（键：符号对，值：频率）
    stats = defaultdict(int)
    # 符号对索引（键：符号对，值：{单词索引: 出现次数}）
    indices = defaultdict(lambda: defaultdict(int))
    # 单词的第一个字符开始，依次与下一个字符组成相邻符号对（如 word=('a','b','c')会生成 (a,b)和 (b,c)
    for i, (word, freq) in enumerate(vocab):
        prev_char = word[0]
        for char in word[1:]:
            stats[prev_char, char] += freq
            # indices[('A', 'B')] = {
            #     0: 5,  # 符号对 (A,B) 在单词0中出现5次（或1次事件，具体取决于实现）
            #     1: 3,  # 符号对 (A,B) 在单词1中出现3次（或1次事件）
            #     2: 2   # 符号对 (A,B) 在单词2中出现2次（或1次事件）
            # }
            indices[prev_char, char][i] += 1
            prev_char = char
    return stats, indices


def replace_pair(pair, vocab, indices):
    """
    用于将词汇表中所有指定符号对（如 ('A', 'B')）的出现替换为合并后的新符号（如 AB
    :param pair: (str,str) 待合并的符号对
    :param vocab: List((word,freq))
    :param indices: {key:pair,val:{idx,freq}}
    :return: 记录所有被修改的单词信息，每个元素为 (j, new_word, word, freq)（j是单词索引，new_word是合并后的新单词，word是原单词，freq是频率）。
    """
    first, second = pair
    # 'AB'
    pair_str = ''.join(pair)
    # 转义反斜杠，避免正则冲突
    pair_str = pair_str.replace('\\', '\\\\')
    # (?<!...)是正向否定回顾后发断言（Negative Lookbehind Assertion），用于检查当前位置之前的字符是否不满足括号内的模式。
    # (?!)是正向否定前瞻断言（Negative Lookahead Assertion），用于检查当前位置之后的字符是否不满足括号内的模式。
    # re.escape :用空格连接后的字符串中的所有正则特殊字符转义
    pattern = re.compile(r'(?<!\S)' + re.escape(first + ' ' + second) + r'(?!\S)')
    iterator = indices[pair].items()
    changes = []
    for j, freq in iterator:
        if freq < 1:
            continue
        word, freq = vocab[j]
        # ('A','B') → 'A B'
        new_word = ''.join(word)
        # 在 string中找到所有匹配 pattern的子串，用 pair_str替换这些子串。
        new_word = pattern.sub(pair_str, new_word)
        # 转回元组形式（如 'AB' → ('AB',)）
        new_word = tuple(new_word.split(' '))

        vocab[j] = (new_word, freq)
        changes.append((j, new_word, word, freq))
    return changes


def prune_stats(stats, big_stats, threshold):
    """
    通过删除频率低于阈值的符号对，减小 stats字典的规模，从而提升 max()函数的效率。同时，通过 big_stats保留被剪枝符号对的实际频率，确保统计信息的完整性。
    :param stats: 符号对频率统计字典（键：符号对，值：频率)，需要被剪枝优化
    :param big_stats: 完整统计字典（键：符号对，值：实际频率），用于保存被剪枝符号对的真实频率。
    :param threshold:频率阈值，低于此值的符号对将被剪枝。
    :return:
    """
    for item, freq in list(stats.items()):
        if freq < threshold:
            del stats[item]
        if freq < 0:
            # 处理负频率
            big_stats[item] += freq
        else:
            big_stats[item] = big_stats[item]


def learn_bpe(infile_names, outfile_name, num_symbols, min_frequency=2, verbose=False, is_dict=False,
              total_symbols=False):
    """

    :param infile_names: List[str] 输入文件路径列表，函数从这些文件中读取语料数据以构建词汇表
    :param out_files_name: str	输出文件路径，函数会将学习到的 BPE 子词规则（合并顺序）写入此文件。
    :param num_symbols: int 需要学习的子词数量（即最终生成的子词词汇表大小）
    :param min_frequency: int 符号对的最小频率阈值。仅当符号对频率 ≥ 此值时，才可能被选为合并对象。
    :param verbose: bool 是否启用详细输出模式。若为 True，函数会打印调试信息（如当前处理的符号对、频率等）。
    :param is_dict: bool 输入文件是否为“字典格式”。若为 True，文件每行需包含“单词 计数”；否则为普通文本。
    :param total_symbols: bool 是否将词内字符和词尾字符视为独立子词。若为 True，会提前扣除这些字符的数量，减少需要学习的子词数量。
    :return:
    """
    # step 1: 强制标准输入/输出/错误流使用 UTF-8 编码
    sys.stderr = codecs.getwriter('UTF-8')(sys.stderr.buffer)
    sys.stdout = codecs.getwriter('UTF-8')(sys.stdout.buffer)
    sys.stdin = codecs.getreader('UTF-8')(sys.stdin.buffer)

    # step 2: 词汇表构建与预处理
    vocab = Counter()
    for f in infile_names:
        sys.stderr.write('Collexting vocab from {}\n'.format(f))
        vocab = update_vocabulary(vocab, f, is_dict)
    # (hello,1) -> (('h', 'e', 'l', 'l', 'o', '</w>'),1)
    vocab = dict([(tuple(x[:-1]) + (x[-1] + '</w>',), y) for (x, y) in vocab.items()])
    sorted_vocab = sorted(vocab.items(), key=lambda x: x[1], reverse=True)

    # step 3: 符号对统计和完整信息备份
    # stats:{('a','b'):1, ('b','c'):1}
    # indices为 {('a','b'):{0:1}, ('b','c'):{1:1}}
    stats, indices = get_pair_statistic(sorted_vocab)
    big_stats = copy.deepcopy(stats)

    # step 4:动态剪枝优化
    if total_symbols:
        uniq_char_internal = set()
        uniq_char_final = set()
        for word in vocab:
            for char in word[:-1]:
                uniq_char_internal.add(char)
            uniq_char_final.add(word[-1])
        num_symbols -= len(uniq_char_internal) + len(uniq_char_final)

    # step 5: 子词合并与规则输出
    sys.stderr.write(f'Write vocab file to {outfile_name}')
    with codecs.open(outfile_name, 'w', encoding='utf-8') as outfile:
        outfile.write('#version: 0.2\n')
        # threshold is inspired by Zipfian assumption, but should only affect speed
        threshold = max(stats.values()) / 10
        for i in range(num_symbols):
            most_frequent = max(stats, key=lambda x: (stats[x], x))
            # we probably missed the best pair because of pruning; go back to full statistics
            if not stats or (i and stats[most_frequent] < threshold):
                prune_stats(stats, big_stats, threshold)
                stats = copy.deepcopy(big_stats)
                most_frequent = max(stats, key=lambda x: (stats[x], x))
                # threshold is inspired by Zipfian assumption, but should only affect speed
                threshold = stats[most_frequent] * i / (i + 10000.0)
                prune_stats(stats, big_stats, threshold)

            if stats[most_frequent] < min_frequency:
                sys.stderr.write(f'no pair has frequency >= {min_frequency}. Stopping\n')
                break

            # 输出模式
            if verbose:
                sys.stderr.write(f'pair{i}:{most_frequent[0]} {most_frequent[1]}  freq{stats[most_frequent]}\n')
            outfile.write(f'{most_frequent[0]} {most_frequent[1]}\n')
            # 合并替换
            changes = replace_pair(most_frequent, sorted_vocab, indices)
            # 更新统计信息
            update_pair_statistics(most_frequent, changes, stats, indices)
            stats[most_frequent] = 0
            if not i % 100:
                prune_stats(stats, big_stats, threshold)

### 9.2 Apply BPE
Use operations learned with learn_bpe.py to encode a new text.

In [23]:
from __future__ import unicode_literals, division
import sys
import os
import inspect
import codecs
import io
import re
import warnings
import random

In [24]:
class BPE(object):

    def __init__(self, codes, merges=-1, separator='@@', vocab=None, glossaries=None):
        """

        :param self:
        :param codes:
        #version: 0.2  # 版本行（行号1）
        a b            # 合并规则1（行号2）
        c d            # 合并规则2（行号3）
        e f            # 合并规则3（行号4）
        g h            # 合并规则4（行号5）
        i j            # 合并规则5（行号6）

        :param merges:
        :param separator:
        :param vocab:
        :param glossaries:
        :return:
        """
        # 将文件指针强制重置回文件的开头位置（位置0）
        codes.seek(0)
        offset = 1

        # check version information
        firstline = codes.readline()
        if firstline.startswith('#version:'):
            # #version: 0.2 -> (0,2)
            self.version = tuple([int(x) for x in re.sub(r'(\.0+)*$', '', firstline.split()[-1]).split(".")])
            offset += 1
        else:
            self.version = (0, 1)
        # [('a', 'b'), ('c', 'd'), ('e', 'f')]
        self.bpe_codes = [tuple(item.strip('\r\n').split(' '))
                          for (n, item) in enumerate(codes)
                          if (n < merges or merges == -1)]
        for i, item in enumerate(self.bpe_codes):
            if len(item) != 2:
                if len(item) != 2:
                    sys.stderr.write(f'Error: invalid line {i + offset} in BPE codes file: {" ".join(item)}\n')
                    sys.stderr.write('The line should exist of exactly two subword units, separated by whitespace\n')
                    sys.exit(1)
            codes.seek(0)
        # 处理重复合并的：{('e', 'f'): 2,('c', 'd'): 1,('a', 'b'): 0} 《- [(2, ('a','b')), (1, ('c','d')), (0, ('a','b'))]
        self.bpe_codes = dict([(code, i) for (i, code) in reversed(list(enumerate(self.bpe_codes)))])
        #{'ab':{('a','b')}}
        self.bpe_codes_reverse = dict([(pair[0] + pair[1], pair) for pair, i in self.bpe_codes.items()])
        # 子词分隔符 un@know->un，known
        self.separator = separator
        self.vocab = vocab
        # 用户指定的术语列表（如["COVID-19", "NLP"]），这些术语在分词时需保持完整，不被拆分。
        self.glossaries = glossaries if glossaries else []
        # ^(COVID-19|NLP)$）
        self.glossaries_regex = re.compile('^({})$'.format('|'.join(self.separator))) if glossaries else None
        # 缓存已处理的编码结果
        self.cache = {}

    def process_line(self, line, dropout=0):
        """
         BPE 分词器中处理单行文本的核心方法，其核心目标是在保留原始行前导/尾随空白符的前提下，对中间内容进行 BPE 分词
        :param self:
        :param line:每一行文本
        :param dropout:
        :return:
        """
        out = ""
        # 前导空白的总长度
        leading_whitespace = len(line) - len(line.lstrip('\r\n '))
        if leading_whitespace:
            # line = "  hello world",out=" "
            out += line[:leading_whitespace]
        #  line = "  hello world"，则 segment处理 "hello world"后可能返回 "h@@ ello w@@ orld"
        out += self.segment(line, dropout)

        trailing_whitespace = len(line) - len(line.rstrip('\r\n '))
        if trailing_whitespace and trailing_whitespace != len(line):
            out += line[-trailing_whitespace:]
        return out

    def segment(self, sentence, dropout=0):
        """
        对外接口，处理完整的文本行（已按空格分词），保留原始空白符并调用 segment_tokens执行实际分词
        :param self:
        :param sentence: 已按空格分割的文本行（如 "hello world"）
        :param dropout:
        :return:
        """
        segments = self.segment_tokens(sentence.strip('\r\n').split(' '), dropout)
        return ' '.join(segments)

    def segment_tokens(self, tokens, dropout=0):
        """
        对每个 token 执行术语隔离、BPE 合并、子词拼接
        :param self:
        :param tokens:  ["hello", "world"]
        :param dropout:
        :return: 子词列表（如 ["h@@", "ello", "w@@", "orld"]
        """
        output = []
        for word in tokens:
            if not word:
                continue
            new_word = [out for segment in self._isolate_glossaries(word)
                        for out in encode(segment,
                                          self.bpe_codes,
                                          self.bpe_codes_reverse,
                                          self.vocab,
                                          self.separator,
                                          self.version,
                                          self.cache,
                                          self.glossaries_regex,
                                          dropout
                                          )]
            for item in new_word[:-1]:
                output.append(item + self.separator)
            output.append(new_word[-1])
            return output

    def _isolate_glossary(self, word):
        """
        隔离术语
        :param self:
        :param word:
        :return:
        """
        word_segments = [word]
        for gloss in self.glossaries:
            word_segments = [out_segments for segment in word_segments
                             for out_segments in isolate_glossary(segment, gloss)]
        return word_segments


def encode(orig, bpe_codes, bpe_codes_reverse, vocab, separator, version, cache, glossaries_regex=None, dropout=0):
    """
    输入的原始单词（orig）通过应用 BPE 合并规则，转换为符合词汇表的子词序列
    :param orig: 待编码的原始单词（如 "hello"）
    :param bpe_codes: 正向合并规则字典 {('e', 'f'): 2,('c', 'd'): 1,('a', 'b'): 0}
    :param bpe_codes_reverse: 反向合并规则字典（键为合并后的子词，值为对应的合并对，如 {"ab": ("a", "b")}）
    :param vocab: 预定义词汇表
    :param separator: 子词分隔符（如 @@）
    :param version: BPE 版本（控制词尾标记的处理方式，如 (0, 1)或 (0, 2)）
    :param cache: 缓存字典（存储已处理单词的编码结果）
    :param glossaries_regex: 术语正则表达式
    :param dropout: 训练时随机丢弃合并对的概率
    :return: 编码后的子词元组（如 ("h@@", "ell", "o")）
    """
    # step 1:缓存与术语处理
    if not dropout and orig in cache:
        return cache[orig]
    if glossaries_regex and glossaries_regex.match(orig):
        cache[orig] = (orig,)
        return (orig,)

    '''
    step 2:初始化词尾标记
    版本0.1：在词尾添加 </w>（如 "hello"→ ['h','e','l','l','o','</w>']）。
    版本0.2：在最后一个字符后添加 </w>（如 "hello"→ ['h','e','l','l','o</w>']），更一致处理词尾段。
    '''
    if version == (0, 1):
        word = list(orig) + ['</w>']
    elif version == (0, 2):
        word = list(orig[:-1]) + [orig[-1] + '</w>']

    # step 3:迭代合并
    while len(word) > 1:
        # 生成候选合并对
        pairs = [
            (bpe_codes[pair], i, pair)  # 生成元组（优先级索引，起始位置，字符对）
            for (i, pair) in enumerate(zip(word, word[1:]))  # 遍历所有相邻字符对
            if (not dropout or random.random() > dropout)  # 过滤 dropout 丢弃的合并对
               and pair in bpe_codes  # 确保合并对存在于 BPE 规则中
        ]
        if not pairs:
            break

        # 选择优先级最高的合并对（最小索引）
        bigram = min(pairs)[2]

        # 确定所有需要合并的位置
        positions = [i for (rank, i, pair) in pairs if pair == bigram]

        # 执行合并操作
        i = 0
        new_word = []
        bigram_str = ''.join(bigram)
        for j in positions:
            # 跳过重叠的合并对（如 (x,x,x) 中的第二个 x）
            if j < i:
                continue
            # 添加未合并部分
            new_word.extend(word[i:j])
            # 添加合并后的子词
            new_word.append(bigram_str)
            i = j + 2
        # 添加剩余未处理部分
        new_word.extend(word[i:])
        word = new_word

        # step 4:词尾处理
        if word[-1] == '</w>':
            word = word[:-1]
        elif word[-1].endswith('</w>'):
            #  ['h','e','l','l','o</w>']→ ['h','e','l','l','o']
            word[-1] = word[-1][:-4]

        # step 5:词汇表检查和拆分
        word = tuple(word)
        if vocab:
            word = check_vocab_and_split(word, bpe_codes_reverse, vocab, separator)

        # cache["hello"] = ('he', 'llo')
        cache[orig] = word
        return word


def recursive_split(segment, bpe_codes_reverse, vocab, separator, final=False):
    """
    反向拆分 OOV 子词：对于输入的子词 segment（可能是 OOV），通过反转 BPE 合并规则，逐步拆分为更小的子单元，直到所有子单元可匹配词汇表或无法继续拆分。
    :param segment: 待拆分的子词（如 "hello"或 "he@@ llo"）。
    :param bpe_codes_reverse: 反向合并规则字典（键为合并后的子词，值为对应的合并对，如 {"ab": ("a", "b")}）
    :param vocab: 预定义词汇表
    :param separator: 子词分隔符（如 @@）
    :param final: 布尔值，标记是否为词的结尾（影响词尾标记的处理）
    :return:
    """
    # step 1:查找反向合并树-通过 bpe_codes_reverse反向查找当前 segment的合并来源
    try:
        if final:
            # 词尾模式：查找 "segment+</w>" 的合并对（键为合并后的子词）
            left, right = bpe_codes_reverse[segment + '</w>']
            # 移除词尾标记的后缀（如 "</w>" 长度为4）
            right = right[:-4]
        else:
            #非词尾模式
            left, right = bpe_codes_reverse[segment]
    except KeyError:
        # 无合并对可查，无法拆分，返回原segment
        yield segment
        return

        # step 2:处理左子单元
    if left + separator in vocab:
        yield left
    else:
        for item in recursive_split(left, bpe_codes_reverse, vocab, separator, final=False):
            yield item

    # step 3:处理右单元
    if (final and right in vocab) or (not final and right + separator in vocab):
        yield right
    else:
        for item in recursive_split(right, bpe_codes_reverse, vocab, separator, final=True):
            yield item


def check_vocab_and_split(orig, bpe_codes_reverse, vocab, separator):
    """
    BPE 分词器中处理词汇表外子词（OOV, Out-of-Vocabulary）
    :param orig: 待处理的子词序列（如 ["h@@", "ell", "o"]）
    :param bpe_codes_reverse: 反向合并规则字典（键为合并后的子词，值为对应的合并对，如 {"ab": ("a", "b")}）
    :param vocab: 预定义词汇表
    :param separator: 子词分隔符（如 @@）
    :return:
    """
    out = []
    # step 1:处理前n-1个子词
    for segment in orig[:-1]:
        # 检查 "子词+分隔符" 是否在词汇表中
        if segment + separator in vocab:
            out.append(segment)
        else:
            # 不存在则递归拆分
            for item in recursive_split(segment, bpe_codes_reverse, vocab, separator, False):
                out.append(item)

    # step 2:处理最后一个子词
    segment = orig[-1]
    if segment in vocab:
        out.append(segment)
    else:
        # 不存在则递归拆分（允许拆分到词尾）
        for item in recursive_split(segment, bpe_codes_reverse, vocab, separator, True):
            out.append(item)
    return out


def read_vocabulary(vocab_file, threshold):
    """
    原始词汇表文件中读取单词及其频率，并保留满足频率阈值的单词
    :param vocab_file: 词汇表文件对象
    :param threshold: 频率阈值
    :return:
    """
    vocabulary = set()
    for line in vocab_file:
        word, freq = line.strip('\r\n').split(' ')
        freq = int(freq)
        if threshold == None or freq >= threshold:
            vocabulary.add(word)
    return vocabulary


def isolate_glossary(self, word, glossary):
    """
    隔离单词中包含的术语表
    :param self: 待处理的原始单词（字符串），可能包含多个术语实例（如 "1934USABUSA"）
    :param word: ：需要隔离的术语（字符串），如 "USA"
    :return:
    """
    # 判断 word是否完全等同于术语 or 判断 word是否包含术语
    if re.match('^' + glossary + '$', word) or not re.search(glossary, word):
        return [word]
    else:
        # re.split(r'(USA)', "1934USABUSA")--> ['1934', 'USA', 'B', 'USA']
        segments = re.split(r'({})'.format(glossary), word)
        segments, ending = segments[:-1], segments[-1]
        # 过滤空串
        segments = list(filter(None, segments))
        # ['1934', 'USA', 'B'] + ['USA'] → ['1934', 'USA', 'B', 'USA']
        return segments + [ending.strip('\r\n ')] if ending != '' else segments